# Benchmark Modeling 01

Now that we've fixed the bug with calculating gene intersection

In [24]:
import pandas as pd 
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go

import sys, os 
sys.path.append('../src')

from model import *
from lightning_train import *

In [7]:
human_data = an.read_h5ad('../data/benchmark/human.h5ad')
mouse_data = an.read_h5ad('../data/benchmark/mouse_clipped.h5ad')

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.



In [11]:
human_data[0:50, :].obs

,index
0,F2S4_160113_027_A01
1,F2S4_160113_027_B01
2,F2S4_160113_027_C01
3,F2S4_160113_027_D01
4,F2S4_160113_027_E01
5,F2S4_160113_027_F01
6,F2S4_160113_027_G01
7,F2S4_160113_028_B01
8,F2S4_160113_028_C01
9,F2S4_160113_028_F01


In [4]:
human_cols = [x.strip().upper() for x in human_data.var.index.values]
mouse_cols = [x.strip().upper() for x in mouse_data.var.index.values]

In [5]:
refgenes = list(set(human_cols).intersection(mouse_cols))
len(refgenes), len(human_cols), len(mouse_cols)

(16519, 50281, 45768)

In [6]:
train_labels = pd.read_csv('../data/benchmark/human_labels/human_MTG_labels_clean.csv')
test_labels = pd.read_csv('../data/benchmark/human_labels/human_A1C_labels_clean.csv')

In [7]:
data_path = '../data/benchmark/'

human = an.read_h5ad(join(data_path, 'human.h5ad'), backed='r+')
mouse = an.read_h5ad(join(data_path, 'mouse_clipped.h5ad'), backed='r+')

human_cols = [x.strip().upper() for x in human.var.index.values]
mouse_cols = [x.strip().upper() for x in mouse.var.index.values]

refgenes = list(set(human_cols).intersection(mouse_cols))

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


## Human Data:  Trained on MTG

In [9]:
module = DataModule(
    datafiles=['../data/benchmark/human.h5ad'],
    labelfiles=['../data/benchmark/human_labels/human_MTG_labels_clean.csv'],
    class_label='subclass_label',
    sep=',',
    batch_size=16,
    index_col='cell',
    num_workers=0,
    deterministic=False,
    normalize=True,
    assume_numeric_label=False,
    stratify=False,
)

model = SIMSClassifier.load_from_checkpoint(
    '../checkpoints/checkpoint-40-desc-human_benchmark_model_no_refgenes_mtg.ckpt',
    input_dim=module.input_dim,
    output_dim=module.output_dim,
)

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.



Initializing network
Initializing explain matrix


In [58]:
labels

['../data/benchmark/human_labels/human_A1C_labels_clean.csv',
 '../data/benchmark/human_labels/human_S1ul_labels_clean.csv',
 '../data/benchmark/human_labels/human_V1C_labels_clean.csv',
 '../data/benchmark/human_labels/human_S1lm_labels_clean.csv',
 '../data/benchmark/human_labels/human_M1ul_labels_clean.csv',
 '../data/benchmark/human_labels/human_M1lm_labels_clean.csv',
 '../data/benchmark/human_labels/human_CgG_labels_clean.csv',
 '../data/benchmark/human_labels/human_MTG_labels_clean.csv']

In [55]:
labels = [os.path.join(f'../data/benchmark/human_labels/{f}') for f in os.listdir('../data/benchmark/human_labels/')]

results = []
for label in labels:
    module = DataModule(
        datafiles=['../data/benchmark/human.h5ad'],
        labelfiles=[label],
        class_label='subclass_label',
        sep=',',
        batch_size=16,
        index_col='cell',
        num_workers=0,
        deterministic=False,
        normalize=True,
        assume_numeric_label=False,
        stratify=False,
    )
    
    module.setup()
#     print(len(module.trainloader.dataset), len(module.testloader.dataset))
    
    trainer = pl.Trainer()
    r = trainer.test(model, module.trainloader)
    results.append(r)

Creating train/val/test DataLoaders...


/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning:

`max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Done, continuing to training.
Calculating weights


/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning:

The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.



Testing: 0it [00:00, ?it/s]

...

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning:

The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.



Testing: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning:

No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score



────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.30809304118156433
 test_balanced_accuracy     0.8413702249526978
  test_confusion_matrix    0.044307779520750046
         test_f1            0.9269029498100281
     test_loss_epoch        0.39436620473861694
     test_median_f1          0.948638838475499
    test_per_class_f1               nan
test_per_class_precision            nan
  test_per_class_recall             nan
     test_precision         0.9269029498100281
       test_recall          0.9269029498100281
    test_specificity        0.9959354400634766
   test_total_accuracy      0.9269029498100281
 test_weighted_accuracy     0.9269029498100281
──────────────────────────────────────────────────────────────────────────

/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:206: RuntimeWarning:

invalid value encountered in true_divide



In [16]:
results[0][0].keys()

dict_keys(['test_loss_epoch', 'test_total_accuracy', 'test_balanced_accuracy', 'test_weighted_accuracy', 'test_precision', 'test_recall', 'test_f1', 'test_per_class_f1', 'test_per_class_precision', 'test_per_class_recall', 'test_specificity', 'test_confusion_matrix', 'test_auroc', 'test_median_f1'])

In [56]:
# import plotly.express as px 

ys = results[0][0].keys()
x = [f.split('/')[-1][:-17].replace('_', ' ').title() for f in labels]
df = pd.DataFrame()

for metric in ys:
    y = [y[0][metric] for y in results]
    df['x'] = x
    df[metric] = y 

In [57]:
df = df.sort_values(by='x')
fig = go.Figure(
    data=[
        go.Bar(
            x=df['x'], 
            y=df['test_balanced_accuracy'], 
            name='Balanced Accuracy',
            text=df['test_balanced_accuracy'].round(2),
            textposition='auto',

        ),
        go.Bar(
            x=df['x'], 
            y=df['test_weighted_accuracy'], 
            name='Weighted Accuracy',
            text=df['test_weighted_accuracy'].round(2),
            textposition='auto',
        )
    ],
    layout=go.Layout(
        title='Weighted and Balanced Accuracy, Test Set (MTG Model)',
        xaxis=dict(title='Dataset'),
        yaxis=dict(title='Scores'),
        font_family="Serif"
    )
)

fig.update_layout(barmode='group')
# fig.write_image('../../ms-thesis/images/ood_metrics/mtg_acc.pdf', scale=3)
fig.show()

In [59]:
fig = go.Figure(
    data=[
        go.Bar(
            x=df['x'], 
            y=df['test_median_f1'], 
            name='Median F1', 
            text=df['test_median_f1'].round(2), 
            textposition='auto'
        ),
        go.Bar(
            x=df['x'], 
            y=df['test_f1'], 
            name='F1',
            text=df['test_f1'].round(2), 
            textposition='auto'
        )
    ],
    layout=go.Layout(
        title='F1 and Median F1 (by class), Test Set (MTG Model)',
        xaxis=dict(title='Dataset'),
        yaxis=dict(title='Scores'),
        font_family="Serif"
    )
)

fig.update_layout(barmode='group')
# fig.write_image('../../ms-thesis/images/ood_metrics/mtg_f1.pdf', scale=3)
fig.show()

## Human Data Trained on V1C

In [42]:
module = DataModule(
    datafiles=['../data/benchmark/human.h5ad'],
    labelfiles=['../data/benchmark/human_labels/human_V1C_labels_clean.csv'],
    class_label='subclass_label',
    sep=',',
    batch_size=16,
    index_col='cell',
    num_workers=0,
    deterministic=False,
    normalize=True,
    assume_numeric_label=False,
    stratify=False,
)

model = SIMSClassifier.load_from_checkpoint(
    '../checkpoints/checkpoint-70-desc-human_benchmark_no_refgenes_v1c.ckpt',
    input_dim=module.input_dim,
    output_dim=module.output_dim,
)

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.



Initializing network
Initializing explain matrix


In [43]:
labels = [os.path.join(f'../data/benchmark/human_labels/{f}') for f in os.listdir('../data/benchmark/human_labels/')]

results = []
for label in labels:
    module = DataModule(
        datafiles=['../data/benchmark/human.h5ad'],
        labelfiles=[label],
        class_label='subclass_label',
        sep=',',
        batch_size=16,
        index_col='cell',
        num_workers=0,
        deterministic=False,
        normalize=True,
        assume_numeric_label=False,
        stratify=False,
    )

    trainer = pl.Trainer()
    r = trainer.test(model, datamodule=module)
    results.append(r)

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning:

`max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


assume_numeric_label=False, using sklearn.preprocessing.LabelEncoder and encoding target variables.
Transforming labelfile 1/1
Creating train/val/test DataLoaders...


/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.



Done, continuing to training.
Calculating weights


/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning:

The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.



...

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning:

The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.



Testing: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning:

No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score



────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.32062244415283203
 test_balanced_accuracy     0.8531653881072998
  test_confusion_matrix    0.044277891516685486
         test_f1            0.9323399066925049
     test_loss_epoch        0.35911309719085693
     test_median_f1         0.9521335807050092
    test_per_class_f1               nan
test_per_class_precision            nan
  test_per_class_recall             nan
     test_precision         0.9323399066925049
       test_recall          0.9323399066925049
    test_specificity        0.9962412118911743
   test_total_accuracy      0.9323399066925049
 test_weighted_accuracy     0.9323399066925049
──────────────────────────────────────────────────────────────────────────

/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:206: RuntimeWarning:

invalid value encountered in true_divide

/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:207: RuntimeWarning:

invalid value encountered in true_divide



In [15]:
ys = results[0][0].keys()
x = [f.split('/')[-1][:-17].replace('_', ' ').title() for f in labels]
df = pd.DataFrame()

for metric in ys:
    y = [y[0][metric] for y in results]
    df['x'] = x
    df[metric] = y 
    
#     fig = px.bar(df, x='x', y='y')
    
#     fig.update_layout(
#         title=f"{metric.replace('_', ' ').title()}, Trained on Human MTG",
#         xaxis_title=f"Dataset",
#         yaxis_title=f"{metric.replace('_', ' ').title()}",
#         font_family="Serif",
#     )
    
#     fig.write_image(f'../../ms-thesis/images/metrics/mtg_{metric}.pdf', scale=3)
#     fig.show()

df

In [44]:
df = df.sort_values(by='test_balanced_accuracy')
fig = go.Figure(
    data=[
        go.Bar(
            x=df['x'], 
            y=df['test_balanced_accuracy'], 
            name='Balanced Accuracy',
            text=df['test_balanced_accuracy'].round(2),
            textposition='auto',

        ),
        go.Bar(
            x=df['x'], 
            y=df['test_weighted_accuracy'], 
            name='Weighted Accuracy',
            text=df['test_weighted_accuracy'].round(2),
            textposition='auto',
        )
    ],
    layout=go.Layout(
        title='Weighted and Balanced Accuracy, Test Set (V1C Model)',
        xaxis=dict(title='Dataset'),
        yaxis=dict(title='Scores'),
        font_family="Serif"
    )
)

fig.update_layout(barmode='group')
fig.write_image('../../ms-thesis/images/ood_metrics/v1c_acc.pdf', scale=3)
fig.show()

In [45]:
df = df.sort_values(by='test_median_f1')
fig = go.Figure(
    data=[
        go.Bar(
            x=df['x'], 
            y=df['test_median_f1'], 
            name='Median F1', 
            text=df['test_median_f1'].round(2), 
            textposition='auto'
        ),
        go.Bar(
            x=df['x'], 
            y=df['test_f1'], 
            name='F1',
            text=df['test_f1'].round(2), 
            textposition='auto'
        )
    ],
    layout=go.Layout(
        title='F1 and Median F1 (by class), Test Set (V1C Model)',
        xaxis=dict(title='Dataset'),
        yaxis=dict(title='Scores'),
        font_family="Serif"
    )
)

fig.update_layout(barmode='group')
fig.write_image('../../ms-thesis/images/ood_metrics/v1c_f1.pdf', scale=3)
fig.show()